In [1]:
from deepface import DeepFace
import cv2
import numpy as np
import sys, time, math
from  collections import deque

In [39]:
#face recognition
dfs = DeepFace.find(img_path = "C:/Users/pierr/Desktop/Telecom_Nancy/3A/PI/biden.jpg",
      db_path = "C:/Users/pierr/.deepface/database", 
      model_name = models[1]
)

Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  3.96it/s]

[{'age': 23, 'region': {'x': 595, 'y': 289, 'w': 165, 'h': 165}, 'gender': {'Woman': 11.393897235393524, 'Man': 88.60610127449036}, 'dominant_gender': 'Man', 'race': {'asian': 22.31120765209198, 'indian': 0.8387289941310883, 'black': 0.11466089636087418, 'white': 58.494848012924194, 'middle eastern': 5.927568674087524, 'latino hispanic': 12.312985211610794}, 'dominant_race': 'white', 'emotion': {'angry': 0.024802189750913663, 'disgust': 1.1586043926504749e-08, 'fear': 96.51091674035291, 'happy': 0.5602003409087861, 'sad': 2.4123472668139496, 'surprise': 0.00038641158000597833, 'neutral': 0.49134500459402825}, 'dominant_emotion': 'fear'}, {'age': 42, 'region': {'x': 191, 'y': 247, 'w': 344, 'h': 344}, 'gender': {'Woman': 0.07657475653104484, 'Man': 99.92343187332153}, 'dominant_gender': 'Man', 'race': {'asian': 92.57705300127829, 'indian': 1.561580132802709, 'black': 0.07476305163496118, 'white': 2.359925301679933, 'middle eastern': 0.22729558023267027, 'latino hispanic': 3.199377274587

Pour avoir age, genre, ethnie, emotion

In [ ]:
dfs = DeepFace.analyze(img_path = "C:/Users/pierr/Desktop/Telecom_Nancy/3A/PI/biden.jpg",
      actions = ['age', 'gender', 'race', 'emotion'] 
)
print(dfs)

Pour trouver visage dans une image

In [16]:
#embeddings
embedding_objs = DeepFace.represent(img_path = "biden.jpg", 
      model_name = models[2]
)
embedding_objs

[{'embedding': [-0.18540459871292114,
   -0.2741902768611908,
   -0.5611828565597534,
   0.0626794770359993,
   -0.17255453765392303,
   0.17704632878303528,
   -0.5100257992744446,
   0.08886171877384186,
   -0.25228574872016907,
   0.5674673914909363,
   0.9514908790588379,
   0.3246096074581146,
   0.18549366295337677,
   -0.7518408894538879,
   0.20624463260173798,
   -0.7521735429763794,
   0.009690184146165848,
   0.44761136174201965,
   0.2781603932380676,
   -0.25882211327552795,
   -0.9279192090034485,
   0.42773932218551636,
   1.1279933452606201,
   -0.9716652631759644,
   -0.44517773389816284,
   0.041002191603183746,
   0.28001511096954346,
   -0.4245935380458832,
   -0.6834816336631775,
   -0.09204522520303726,
   -0.151948943734169,
   0.8119856715202332,
   -0.022492622956633568,
   0.3016563057899475,
   -0.11338628828525543,
   -0.3193762004375458,
   0.1703873872756958,
   0.020438093692064285,
   -0.5409654378890991,
   -0.05893446505069733,
   -0.11543981730937958,

In [22]:
class FPS (object):
    def __init__(self,avarageof=50):
        self.frametimestamps = deque(maxlen=avarageof)
    def __call__(self):
        self.frametimestamps.append(time.time())
        if(len(self.frametimestamps) > 1):
            return len(self.frametimestamps)/ \
                (self.frametimestamps[-1]-self.frametimestamps[0])
        else:
            return 0.0

In [26]:
class TextWriter(object) : 
    def __init__(self):
        self._font                   = cv2.FONT_HERSHEY_SIMPLEX
        self._fontScale              = 1
        self._fontColor              = (0,0,255)
        self._lineType               = 2
    def __call__(self, img, text, pos=(40,40)) :
        cv2.putText(img, text, pos, 
                    self._font, 
                    self._fontScale,
                    self._fontColor,
                    self._lineType)

In [2]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

Test avec la caméra

In [42]:
video_capture = cv2.VideoCapture(0)


# Initialize some variables
face_locations = []
process_this_frame = True

fps = FPS()
writer = TextWriter()

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        try :
            face_locations = DeepFace.represent(frame, model_name = models[2])
        except :
            face_locations = []

        
    process_this_frame = not process_this_frame
    if len(face_locations) != 0 :
        for k in range(len(face_locations)):
            x,y,w,h = face_locations[k]['facial_area']['x'], face_locations[k]['facial_area']['y'], face_locations[k]['facial_area']['w'], face_locations[k]['facial_area']['h']
            
            # Draw a box around the face
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0, 0, 255), 2)


    writer(frame, "Fps={:06.2f}".format(fps()))
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  3.92it/s]


[{'age': 33, 'region': {'x': 266, 'y': 227, 'w': 161, 'h': 161}, 'gender': {'Woman': 2.609534002840519, 'Man': 97.39046692848206}, 'dominant_gender': 'Man', 'race': {'asian': 0.37233864422887564, 'indian': 0.2029109513387084, 'black': 0.026177649851888418, 'white': 86.93047165870667, 'middle eastern': 7.342826575040817, 'latino hispanic': 5.125274881720543}, 'dominant_race': 'white', 'emotion': {'angry': 3.4972520476875706e-05, 'disgust': 7.141924061297784e-12, 'fear': 1.1806895720975916e-06, 'happy': 59.779123054690544, 'sad': 0.004549637649838636, 'surprise': 1.7488743940718743e-06, 'neutral': 40.216293347853025}, 'dominant_emotion': 'happy'}]


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.32it/s]


[{'age': 33, 'region': {'x': 258, 'y': 232, 'w': 166, 'h': 166}, 'gender': {'Woman': 1.336824893951416, 'Man': 98.66318106651306}, 'dominant_gender': 'Man', 'race': {'asian': 2.121231332421303, 'indian': 1.9368888810276985, 'black': 0.34852928947657347, 'white': 62.210243940353394, 'middle eastern': 20.429787039756775, 'latino hispanic': 12.953314185142517}, 'dominant_race': 'white', 'emotion': {'angry': 0.035759195452556014, 'disgust': 4.8691733978145635e-08, 'fear': 0.001132572469941806, 'happy': 10.853078216314316, 'sad': 0.05624938057735562, 'surprise': 0.00011497887726363842, 'neutral': 89.05366659164429}, 'dominant_emotion': 'neutral'}]


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.44it/s]


...

[{'age': 31, 'region': {'x': 264, 'y': 234, 'w': 170, 'h': 170}, 'gender': {'Woman': 0.853734090924263, 'Man': 99.14626479148865}, 'dominant_gender': 'Man', 'race': {'asian': 0.1383487251587212, 'indian': 0.23915302008390427, 'black': 0.02254125865874812, 'white': 87.20142841339111, 'middle eastern': 7.6518625020980835, 'latino hispanic': 4.7466691583395}, 'dominant_race': 'white', 'emotion': {'angry': 0.0004849647046127274, 'disgust': 5.592354170288725e-11, 'fear': 0.00017416039000979738, 'happy': 6.57835343947133, 'sad': 0.02306808302072558, 'surprise': 0.00010211526313671665, 'neutral': 93.3978196023595}, 'dominant_emotion': 'neutral'}]


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.49it/s]


[{'age': 34, 'region': {'x': 268, 'y': 238, 'w': 164, 'h': 164}, 'gender': {'Woman': 0.6268716417253017, 'Man': 99.37312006950378}, 'dominant_gender': 'Man', 'race': {'asian': 0.45877345837652683, 'indian': 0.7621093653142452, 'black': 0.1176855992525816, 'white': 81.0845136642456, 'middle eastern': 10.703063756227493, 'latino hispanic': 6.873849779367447}, 'dominant_race': 'white', 'emotion': {'angry': 2.0231635033951534e-06, 'disgust': 2.4797228344048544e-12, 'fear': 1.1522934606489343e-06, 'happy': 92.22737550735474, 'sad': 0.0008319617336383089, 'surprise': 2.7899934806896454e-06, 'neutral': 7.771787792444229}, 'dominant_emotion': 'happy'}]


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.24it/s]


[{'age': 34, 'region': {'x': 264, 'y': 237, 'w': 169, 'h': 169}, 'gender': {'Woman': 1.0762237943708897, 'Man': 98.92377853393555}, 'dominant_gender': 'Man', 'race': {'asian': 3.6750558763742447, 'indian': 2.9159927740693092, 'black': 0.5028811283409595, 'white': 55.09709119796753, 'middle eastern': 23.120875656604767, 'latino hispanic': 14.688101410865784}, 'dominant_race': 'white', 'emotion': {'angry': 0.009277123172068968, 'disgust': 1.2137547043822305e-09, 'fear': 0.001491889725002693, 'happy': 75.04859566688538, 'sad': 0.01975512132048607, 'surprise': 9.810875667426444e-06, 'neutral': 24.920867383480072}, 'dominant_emotion': 'happy'}]


Test sur une vidéo

In [3]:
video = cv2.VideoCapture('C:/Users/pierr/Desktop/Telecom_Nancy/3A/PI/Vidéos test/Crowd walking forward NYC B-Roll.mp4')

# Initialize some variables
face_locations = []
process_this_frame = True

#fps = FPS()
#writer = TextWriter()

frame_width = int(video.get(3)) 
frame_height = int(video.get(4))
size = (frame_width, frame_height)
result = cv2.VideoWriter('C:/Users/pierr/Desktop/Telecom_Nancy/3A/PI/Vidéos test/Result/Crowd walking forward NYC B-Roll_OpenFace.mp4',  
                         cv2.VideoWriter_fourcc(*'mp4v'), 
                         30, size) 

ret = True

while ret :
    # Grab a single frame of video
    ret, frame = video.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        try :
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        except :
            ret, frame = video.read()

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        try :
            face_locations_0 = DeepFace.represent(frame, model_name = models[0])
            face_locations_1 = DeepFace.represent(frame, model_name = models[1])
            face_locations_2 = DeepFace.represent(frame, model_name = models[2])
            face_locations_3 = DeepFace.represent(frame, model_name = models[3])
            face_locations_4 = DeepFace.represent(frame, model_name = models[4])
            face_locations_5 = DeepFace.represent(frame, model_name = models[5])
            face_locations_6 = DeepFace.represent(frame, model_name = models[6])
            face_locations_7 = DeepFace.represent(frame, model_name = models[7])
            face_locations_8 = DeepFace.represent(frame, model_name = models[8])
        except :
            face_locations = []

        if face_locations_0 == face_locations_1 == face_locations_2 == face_locations_3 == face_locations_4 == face_locations_5 == face_locations_6 == face_locations_7 == face_locations_8 :
            print("Les modèles trouvent les mêmes visages")
        else :
            print(face_locations_0, face_locations_1, face_locations_2, face_locations_3, face_locations_4, face_locations_5, face_locations_6, face_locations_7, face_locations_8)
        

    process_this_frame = not process_this_frame

    if len(face_locations) != 0 :
        for k in range(len(face_locations)):
            x,y,w,h = face_locations[k]['facial_area']['x'], face_locations[k]['facial_area']['y'], face_locations[k]['facial_area']['w'], face_locations[k]['facial_area']['h']
            
            # Draw a box around the face
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0, 0, 255), 2)


    #writer(frame, "Fps={:06.2f}".format(fps()))
    result.write(frame) 
    # Display the resulting image
    try:
        cv2.imshow('Video', frame)
    except:
        pass
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video.release()
result.release()
cv2.destroyAllWindows()

NameError: name 'small_frame' is not defined